<a href="https://colab.research.google.com/github/1900690/yolo-traking/blob/main/yolo_traking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#果実トラッキング

※学習前のＡＩで追跡を行う。追跡できる品目はリンゴ、オレンジなど代表的なもののみ

In [1]:
#@title 分析する動画をアップロードまたはサンプル動画(ユズ)で分析

#事前準備
!pip -q install ultralytics

from google.colab import files
import os
サンプル動画を使用する = True # @param {type:"boolean"}

if サンプル動画を使用する == True:
  if os.path.exists('/content/yuzu.mp4'):
     os.remove('/content/yuzu.mp4')
  !wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/yuzu.mp4

else:
  uploaded_file = files.upload()
  uploaded_file_name = next(iter(uploaded_file))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.5/870.5 kB 7.1 MB/s eta 0:00:00


<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/yuzu_before.jpg?raw=1" width=80%>
  <br>
  ユズのサンプル動画
</p>

In [2]:
#@markdown 分析して動画をダウンロード
from ultralytics import YOLO
import shutil
import os
from google.colab import files

if os.path.exists('/content/runs'):
    shutil.rmtree('/content/runs')

# Load a model
model = YOLO("yolo11n.pt")

if サンプル動画を使用する == True:
  results = model.track(source="/content/yuzu.mp4",stream=False,save=True)
  files.download("/content/runs/detect/track/yuzu.avi")
else:
  results = model.track(source="/content/"+uploaded_file_name,stream=False,save=True)
  files.download("/content/runs/detect/track/"+uploaded_file_name[:-4]+".avi")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 68.7MB/s]


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.2s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/210) /content/yuzu.mp4: 384x640 2 apples, 4 oranges, 385.1ms
video 1/1 (frame 2/210) /content/yuzu.mp4: 384x640 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/yuzu_after.jpg?raw=1" width=80%>
  <br>
  果実数を検出後のユズのサンプル動画
</p>